# Ugesco : enrich and desambiguate the Samnang-Hans' JSON

In [2]:
from ugesco import *

Parse the json from Google Drive. TODO : replace this temporary  jq parser with a pure Python script

%%cmd
jq -r ".[] | {beeldid, spatial : .spatial.value[]?, key : .spatial.key[]?, class1:.imageclassification.class1?, prob1:.imageclassification.prob1?, class2:.imageclassification.class2?, prob2:.imageclassification.prob2?} | [.beeldid, .spatial, .key, .class1, .prob1, .class2, .prob2] | @csv" data/jsondata_ugesco.json jq -r ".[] | {beeldid, spatial : .spatial.value[]?, key : .spatial.key[]?, class1:.imageclassification.class1?, prob1:.imageclassification.prob1?, class2:.imageclassification.class2?, prob2:.imageclassification.prob2?} | [.beeldid, .spatial, .key, .class1, .prob1, .class2, .prob2] | @csv" data/jsondata_ugesco.json > data/jsondata.csv 


In [3]:
file = "data/jsondata_ugesco.json"
data = json_to_df(file)

In [1]:
data.head()

NameError: name 'data' is not defined

In [5]:
#Drop temporal and void columns
to_drop = [c for c in data.columns if c.startswith('temporal') or c.endswith('spatial')]
data.drop(to_drop, axis=1, inplace=True)

In [6]:
#remove classes and probs 3 to 5
data.drop(list(data.filter(regex = '(_class|_prob)[3-5]$')), axis = 1, inplace = True)

In [ ]:
#Delete "indoor", "outdoor" etc in class names and replace undescores with spaces
data['imageclassification_class1'] = data['imageclassification_class1'].str.replace("/.+", "").str.replace('_', ' ')
data['imageclassification_class2'] = data['imageclassification_class2'].str.replace("/.+", "").str.replace('_', ' ')
data.head()

In [ ]:
#import de Samnang's csv enriched
class_places = pd.read_csv("data/classes_places.csv", encoding="utf8")
class_places.head()

In [ ]:
# join/merge with classes_places.csv to get the class_ids
data = pd.merge(data, class_places[['class_id', 'original_class']], how='left', left_on=['imageclassification_class1'], right_on=['original_class'])
data = data.rename(columns={'class_id': 'class_id1'})

In [ ]:
data = pd.merge(data, class_places[['class_id', 'original_class']], how='left', left_on=['imageclassification_class2'], right_on=['original_class'], suffixes=['', '_x'])
data = data.rename(columns={'class_id': 'class_id2'})

In [ ]:
to_drop = [c for c in data.columns if c.startswith('original_')]
data.drop(to_drop, axis=1, inplace=True)

In [12]:
#join/merge with phototheque_pallas to get the locations. 
#The csv is gzipped to get around the 100MB file weigth limitation of Github
phototeque = pd.read_csv("data/phototheque_pallas.csv.gz", compression="gzip", encoding="utf8", dtype={'beeldid': str})

data = pd.merge(data, phototeque, how='left', on=['beeldid'], suffixes=['', '_x'])

In [13]:
# Pictures that contains more than a loc in the thesaurus descriptors

more_than_one_loc = data[data['loc_qid'].str.contains(",", na = False)]

In [14]:
#test Rosette API. Max 10 000 calls a month and 1000 a day
        
#data['rosette'] = data['LEGEND'].apply(rosette)

In [15]:
# Some descriptive stats on columns
data.describe(include="all")

,beeldid,imageclassification_class1,imageclassification_class2,imageclassification_prob1,imageclassification_prob2,spatial_key_0,spatial_key_1,spatial_key_2,spatial_key_3,spatial_key_4,...,spatial_value_7,class_id1,class_id2,lien_phototheque,LEGEND,BPALLASF,country_name,country_qid,loc_wikiname,loc_qid
count,735,735,735,735.000000,735.000000,655,326,120,32,15,...,3,735,735,735,735,735,615,615,615,613
unique,735,17,100,NaN,NaN,8,8,7,6,4,...,2,17,97,735,735,329,8,8,78,79
top,00035695,plaza,street,NaN,NaN,LOC_PLACE,LOC_PLACE,LOC_PLACE,LOC_PLACE,LOC_PLACE,...,Bruxelles,Q174782,Q79007,http://pallas.cegesoma.be/pls/opac/opac.search...,"Les funérailles de M. Adolphe Max, Bourgmestre...","Libération de Verviers par les alliés, 9/1944....",Belgique,Q31,Bruxelles,Q239
freq,1,249,73,NaN,NaN,431,183,66,15,10,...,2,249,73,1,1,35,578,578,191,191
mean,NaN,NaN,NaN,0.447857,0.154413,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,0.218975,0.078806,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,0.064885,0.002799,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,0.267549,0.099692,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,0.400570,0.143618,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,0.612765,0.204018,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# More data profiling
import pandas_profiling
pandas_profiling.ProfileReport(data)

Number of variables,30
Number of observations,735
Total Missing (%),45.0%
Total size in memory,178.0 KiB
Average record size in memory,248.0 B
Numeric,2
Categorical,25
Boolean,0
Date,0
Text (Unique),3
Rejected,0


In [17]:
#data.to_csv("C:/Users/ettor/Desktop/ugesco_file_temp.csv", encoding="utf-8")

In [18]:
#apply the algorithm
for index, rows in data.iterrows():
    dic = {}
    dic['index'] = index
    dic['beeldid'] = rows.beeldid
    if ("," not in str(rows.loc_wikiname) 
        and str(rows.loc_wikiname) != "nan"
        and isQidACity(rows.loc_qid)): 
        dic['city_thes_name'] = rows.loc_wikiname
        dic['city_qid_name'] = rows.loc_qid
        #dic['country_name'] = rows.country_name
        #dic['country_qid'] = rows.country_qid
    #elif (rows.spatial_key_0 == "LOC_PLACE" 
          #and isStringACity(rows.spatial_value_0)):
        #dic['city'] = rows.spatial_value_0
    print(dic)

          
        

{'index': 0, 'beeldid': '00009500', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 1, 'beeldid': '00009520', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 2, 'beeldid': '00009537', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 3, 'beeldid': '00009544', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 4, 'beeldid': '00009549', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 5, 'beeldid': '00009551', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 6, 'beeldid': '00009705', 'city': 'Gand', 'city_qid': 'Q1296'}
{'index': 7, 'beeldid': '00009712', 'city': 'Gand', 'city_qid': 'Q1296'}
{'index': 8, 'beeldid': '00009732', 'city': 'Gand', 'city_qid': 'Q1296'}
{'index': 9, 'beeldid': '00009736', 'city': 'Gand', 'city_qid': 'Q1296'}
{'index': 10, 'beeldid': '00009746'}
{'index': 11, 'beeldid': '00009764'}
{'index': 12, 'beeldid': '00009770'}
{'index': 13, 'beeldid': '00009778'}
{'index': 14, 'beeldid': '00009848'}
{'index': 15, 'beeldid': '00009967'}
{'index': 16, 'beeldid':

{'index': 144, 'beeldid': '00033506'}
{'index': 145, 'beeldid': '00033511'}
{'index': 146, 'beeldid': '00033554'}
{'index': 147, 'beeldid': '00033556'}
{'index': 148, 'beeldid': '00033568'}
{'index': 149, 'beeldid': '00033596'}
{'index': 150, 'beeldid': '00033614'}
{'index': 151, 'beeldid': '00033632'}
{'index': 152, 'beeldid': '00033643'}
{'index': 153, 'beeldid': '00033693'}
{'index': 154, 'beeldid': '00033938'}
{'index': 155, 'beeldid': '00033983'}
{'index': 156, 'beeldid': '00034021'}
{'index': 157, 'beeldid': '00034024'}
{'index': 158, 'beeldid': '00034300'}
{'index': 159, 'beeldid': '00035257', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 160, 'beeldid': '00035265', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 161, 'beeldid': '00035470'}
{'index': 162, 'beeldid': '00035473'}
{'index': 163, 'beeldid': '00035480'}
{'index': 164, 'beeldid': '00035568'}
{'index': 165, 'beeldid': '00035586'}
{'index': 166, 'beeldid': '00035613'}
{'index': 167, 'beeldid': '00035625', 'city'

{'index': 319, 'beeldid': '00005539', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 320, 'beeldid': '00005541', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 321, 'beeldid': '00005542', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 322, 'beeldid': '00005544', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 323, 'beeldid': '00005547', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 324, 'beeldid': '00005548', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 325, 'beeldid': '00005549', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 326, 'beeldid': '00005552', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 327, 'beeldid': '00005556', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 328, 'beeldid': '00005557', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 329, 'beeldid': '00005558', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 330, 'beeldid': '00005569', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 331, 'beeldid': '00005572', 'city': 'Bruxe

{'index': 453, 'beeldid': '00014417', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 454, 'beeldid': '00014418', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 455, 'beeldid': '00014420', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 456, 'beeldid': '00014501', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 457, 'beeldid': '00014575', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 458, 'beeldid': '00015666'}
{'index': 459, 'beeldid': '00015714'}
{'index': 460, 'beeldid': '00016741', 'city': 'Anvers', 'city_qid': 'Q12892'}
{'index': 461, 'beeldid': '00017136', 'city': 'Anvers', 'city_qid': 'Q12892'}
{'index': 462, 'beeldid': '00017223', 'city': 'Anvers', 'city_qid': 'Q12892'}
{'index': 463, 'beeldid': '00019056', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 464, 'beeldid': '00020063'}
{'index': 465, 'beeldid': '00020807'}
{'index': 466, 'beeldid': '00021220', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 467, 'beeldid': '00022078', 'city': 'Bruxelles', 'c

{'index': 600, 'beeldid': '00033818'}
{'index': 601, 'beeldid': '00033823'}
{'index': 602, 'beeldid': '00033866'}
{'index': 603, 'beeldid': '00034034'}
{'index': 604, 'beeldid': '00034411', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 605, 'beeldid': '00034417'}
{'index': 606, 'beeldid': '00034423', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 607, 'beeldid': '00034523', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 608, 'beeldid': '00034529', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 609, 'beeldid': '00034543', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 610, 'beeldid': '00034547', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 611, 'beeldid': '00034556', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 612, 'beeldid': '00034558', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 613, 'beeldid': '00034560', 'city': 'Bruxelles', 'city_qid': 'Q239'}
{'index': 614, 'beeldid': '00034589', 'city': 'Anvers', 'city_qid': 'Q12892'}
{'index': 615, 'beeld

In [19]:
data

,beeldid,imageclassification_class1,imageclassification_class2,imageclassification_prob1,imageclassification_prob2,spatial_key_0,spatial_key_1,spatial_key_2,spatial_key_3,spatial_key_4,...,spatial_value_7,class_id1,class_id2,lien_phototheque,LEGEND,BPALLASF,country_name,country_qid,loc_wikiname,loc_qid
0,00009500,mosque,synagogue,0.313512,0.242379,LOC_PLACE,LOC_BUILDING,NaN,NaN,NaN,...,NaN,Q32815,Q34627,http://pallas.cegesoma.be/pls/opac/opac.search...,Brussel. L'église St.e. Marie. [Sipho],Vues de Bruxelles : 1942-1944. - 124 photos,Belgique,Q31,Bruxelles,Q239
1,00009520,building facade,hotel,0.203420,0.184267,LOC_PLACE,LOC_STREET,NaN,NaN,NaN,...,NaN,Q41176,Q27686,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles. Grand' Place.,Vues de Bruxelles : 1942-1944. - 124 photos,Belgique,Q31,Bruxelles,Q239
2,00009537,palace,mansion,0.334647,0.233139,LOC_PLACE,LOC_BUILDING,NaN,NaN,NaN,...,NaN,Q16560,Q1802963,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles. La Château de Tervuren. [Sipho],Vues de Bruxelles : 1942-1944. - 124 photos,Belgique,Q31,Bruxelles,Q239
3,00009544,library,museum,0.706734,0.091893,LOC_PLACE,LOC_SPACES,NaN,NaN,NaN,...,NaN,Q856584,Q24699794,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles. Place de la Monnaie. [Sipho],Vues de Bruxelles : 1942-1944. - 124 photos,Belgique,Q31,Bruxelles,Q239
4,00009549,street,promenade,0.554976,0.219579,LOC_PLACE,LOC_BUILDING,LOC_STREET,NaN,NaN,...,NaN,Q79007,Q787113,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles. Vue du Palais de Justice pris du Pa...,Vues de Bruxelles : 1942-1944. - 124 photos,Belgique,Q31,Bruxelles,Q239
5,00009551,plaza,embassy,0.184917,0.146900,LOC_PLACE,LOC_BUILDING,NaN,NaN,NaN,...,NaN,Q174782,Q2750585,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles - La gare du Nord. [Sipho],Vues de Bruxelles : 1942-1944. - 124 photos,Belgique,Q31,Bruxelles,Q239
6,00009705,church,palace,0.235078,0.206280,LOC_PLACE,NaN,NaN,NaN,NaN,...,NaN,Q16970,Q16560,http://pallas.cegesoma.be/pls/opac/opac.search...,Gand. Hôtel des Postes. [Frei gegeben durch ze...,"Vues de Gent, 1942-1943. - 54 photos",Belgique,Q31,Gand,Q1296
7,00009712,plaza,courthouse,0.151032,0.114809,LOC_BUILDING,NaN,NaN,NaN,NaN,...,NaN,Q174782,Q1137809,http://pallas.cegesoma.be/pls/opac/opac.search...,Eglise Saint-Nicolas et le monument de Georges...,"Vues de Gent, 1942-1943. - 54 photos",Belgique,Q31,Gand,Q1296
8,00009732,street,plaza,0.274491,0.186870,LOC_PLACE,NaN,NaN,NaN,NaN,...,NaN,Q79007,Q174782,http://pallas.cegesoma.be/pls/opac/opac.search...,Gand. Maison des Bouchers. [Sipho],"Vues de Gent, 1942-1943. - 54 photos",Belgique,Q31,Gand,Q1296
9,00009736,plaza,stage,0.147155,0.135074,LOC_PLACE,LOC_MONUMENT,LOC_PLACE,NaN,NaN,...,NaN,Q174782,Q194428,http://pallas.cegesoma.be/pls/opac/opac.search...,"Gand, 4e Ville du Royaume est à la fois une vi...","Vues de Gent, 1942-1943. - 54 photos",Belgique,Q31,Gand,Q1296
